In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import joblib

products = pd.read_pickle('final_cleaned_products.pkl')
products = products.dropna(subset='long')
products.drop(['location'], axis=1, inplace=True)
# Drop the cat0 fields to test the model with cat1 values
# The model was better with cat0 only
# products = products[products.columns.drop(list(products.filter(regex='cat_1')))]
products


,id,price_gbp,long,lat,cat_0__Baby & Kids Stuff,"cat_0__Clothes, Footwear & Accessories",cat_0__Computers & Software,cat_0__DIY Tools & Materials,cat_0__Health & Beauty,cat_0__Home & Garden,...,cat_1__ Washer Dryers,cat_1__ Washing Machines,cat_1__ Watches,cat_1__ Water Sports,cat_1__ Wedding Clothes & Accessories,cat_1__ Winter Sports,cat_1__ Women's Accessories,cat_1__ Women's Clothing,cat_1__ Women's Shoes,cat_1__ Wood & Timber
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,5.0,51.453489,-1.031873,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,20.0,57.479012,-4.225739,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,20.0,53.150228,0.329093,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,59948726-29be-4b35-ade5-bb2fd7331856,115.0,51.291949,-2.447623,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,16dbc860-696e-4cda-93f6-4dd4926573fb,450.0,53.568355,-2.026164,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7049,06d8a5c3-4f22-4d1b-ae7e-6ded471eb775,550.0,53.453547,-2.734323,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7050,cce6d412-bac0-489e-9fba-cfdeeb756117,69.0,53.450693,-2.994883,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7051,88d2d66b-2685-46b8-af84-f495fd2ccb14,380.0,53.450693,-2.994883,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7052,8ca91ce8-49e7-4746-b06c-ac838d94ef35,650.0,53.485152,-2.898906,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Drop more outliers for testing
In the clean products process I made the intially made the decision to remove specific outliers. Below i reintroduced my tabular clenase class to allow me to make more strict outlier removal decisions.


In [29]:
from clean_tabular import TabularCleanse
cleanse = TabularCleanse('fb_marketplace_conn.json', 'aicore2022!')
products = cleanse.remove_rows_conditonal(products, 'price_gbp', 1000.00, '<')
products = cleanse.remove_rows_conditonal(products, 'price_gbp', 0.00, '>')
products

,id,price_gbp,long,lat,cat_0__Baby & Kids Stuff,"cat_0__Clothes, Footwear & Accessories",cat_0__Computers & Software,cat_0__DIY Tools & Materials,cat_0__Health & Beauty,cat_0__Home & Garden,...,cat_1__ Washer Dryers,cat_1__ Washing Machines,cat_1__ Watches,cat_1__ Water Sports,cat_1__ Wedding Clothes & Accessories,cat_1__ Winter Sports,cat_1__ Women's Accessories,cat_1__ Women's Clothing,cat_1__ Women's Shoes,cat_1__ Wood & Timber
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,5.0,51.453489,-1.031873,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,20.0,57.479012,-4.225739,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,20.0,53.150228,0.329093,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,59948726-29be-4b35-ade5-bb2fd7331856,115.0,51.291949,-2.447623,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,16dbc860-696e-4cda-93f6-4dd4926573fb,450.0,53.568355,-2.026164,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7049,06d8a5c3-4f22-4d1b-ae7e-6ded471eb775,550.0,53.453547,-2.734323,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7050,cce6d412-bac0-489e-9fba-cfdeeb756117,69.0,53.450693,-2.994883,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7051,88d2d66b-2685-46b8-af84-f495fd2ccb14,380.0,53.450693,-2.994883,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7052,8ca91ce8-49e7-4746-b06c-ac838d94ef35,650.0,53.485152,-2.898906,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Clustering longitude and latitude data
As longitude and latitude data is complex data i wanted to attempt to use clustering on this data to idenify whther it had a postive impact on model performance. In practice, 3 clusters were identified and the impact on the model was minimal.


In [22]:
cluster_X = products.loc[:,['id', 'long', 'lat']]
print(cluster_X)

# Determine number of cluster to test
K_clusters = range(1,10)
# Create clusters of each size
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
# Plot the cluster gain for each cluster to determine which is preferred
Y_axis = products[['lat']]
X_axis = products[['long']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

                                        id       long       lat
0     243809c0-9cfc-4486-ad12-3b7a16605ba9  51.453489 -1.031873
1     1c58d3f9-8b93-47ea-9415-204fcc2a22e6  57.479012 -4.225739
2     860673f1-57f6-47ba-8d2f-13f9e05b8f9a  53.150228  0.329093
3     59948726-29be-4b35-ade5-bb2fd7331856  51.291949 -2.447623
4     16dbc860-696e-4cda-93f6-4dd4926573fb  53.568355 -2.026164
...                                    ...        ...       ...
7049  06d8a5c3-4f22-4d1b-ae7e-6ded471eb775  53.453547 -2.734323
7050  cce6d412-bac0-489e-9fba-cfdeeb756117  53.450693 -2.994883
7051  88d2d66b-2685-46b8-af84-f495fd2ccb14  53.450693 -2.994883
7052  8ca91ce8-49e7-4746-b06c-ac838d94ef35  53.485152 -2.898906
7053  df8ef910-03cc-4c9e-97a9-7f0a7e838102  51.272337 -0.721647

[6536 rows x 3 columns]


/tmp/ipykernel_12888/3031219528.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [15]:
# Assign all rows to a cluster
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(cluster_X[cluster_X.columns[1:3]]) # Compute k-means clustering.
cluster_X['cluster_long_lat'] = kmeans.fit_predict(cluster_X[cluster_X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(cluster_X[cluster_X.columns[1:3]]) # Labels of each point

cluster_X.plot.scatter(x = 'lat', y = 'long', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
cluster_X = cluster_X.drop(['long', 'lat'], axis=1)


In [16]:
# Merge the cluster back to the products data
products_clustered = products.merge(cluster_X, left_on='id', right_on='id')
# Remove long_lat fields as now defined within clusters
products_clustered = products_clustered.drop(['long', 'lat'], axis=1)
products_clustered

,id,price_gbp,cat_0__Baby & Kids Stuff,"cat_0__Clothes, Footwear & Accessories",cat_0__Computers & Software,cat_0__DIY Tools & Materials,cat_0__Health & Beauty,cat_0__Home & Garden,"cat_0__Music, Films, Books & Games",cat_0__Office Furniture & Equipment,...,cat_1__ Washing Machines,cat_1__ Watches,cat_1__ Water Sports,cat_1__ Wedding Clothes & Accessories,cat_1__ Winter Sports,cat_1__ Women's Accessories,cat_1__ Women's Clothing,cat_1__ Women's Shoes,cat_1__ Wood & Timber,cluster_long_lat
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,5.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,20.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,20.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,59948726-29be-4b35-ade5-bb2fd7331856,115.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,16dbc860-696e-4cda-93f6-4dd4926573fb,450.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,06d8a5c3-4f22-4d1b-ae7e-6ded471eb775,550.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6532,cce6d412-bac0-489e-9fba-cfdeeb756117,69.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6533,88d2d66b-2685-46b8-af84-f495fd2ccb14,380.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6534,8ca91ce8-49e7-4746-b06c-ac838d94ef35,650.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
# Create data for modelling
products_clustered = products_clustered.drop(['id'], axis=1)
X = products_clustered.drop(['price_gbp'], axis=1)
y = products_clustered['price_gbp']
X

,cat_0__Baby & Kids Stuff,"cat_0__Clothes, Footwear & Accessories",cat_0__Computers & Software,cat_0__DIY Tools & Materials,cat_0__Health & Beauty,cat_0__Home & Garden,"cat_0__Music, Films, Books & Games",cat_0__Office Furniture & Equipment,cat_0__Other Goods,"cat_0__Phones, Mobile Phones & Telecoms",...,cat_1__ Washing Machines,cat_1__ Watches,cat_1__ Water Sports,cat_1__ Wedding Clothes & Accessories,cat_1__ Winter Sports,cat_1__ Women's Accessories,cat_1__ Women's Clothing,cat_1__ Women's Shoes,cat_1__ Wood & Timber,cluster_long_lat
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Create data for modelling
If using the clustering method for long_lat this can be skipped.


In [30]:
products = products.drop(['id'], axis=1)

X = products.drop(['price_gbp'], axis=1)
y = products['price_gbp']
X

,long,lat,cat_0__Baby & Kids Stuff,"cat_0__Clothes, Footwear & Accessories",cat_0__Computers & Software,cat_0__DIY Tools & Materials,cat_0__Health & Beauty,cat_0__Home & Garden,"cat_0__Music, Films, Books & Games",cat_0__Office Furniture & Equipment,...,cat_1__ Washer Dryers,cat_1__ Washing Machines,cat_1__ Watches,cat_1__ Water Sports,cat_1__ Wedding Clothes & Accessories,cat_1__ Winter Sports,cat_1__ Women's Accessories,cat_1__ Women's Clothing,cat_1__ Women's Shoes,cat_1__ Wood & Timber
0,51.453489,-1.031873,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,57.479012,-4.225739,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53.150228,0.329093,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,51.291949,-2.447623,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53.568355,-2.026164,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7049,53.453547,-2.734323,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7050,53.450693,-2.994883,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7051,53.450693,-2.994883,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7052,53.485152,-2.898906,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Model(s) intialisation
Here each model type can be intialised


In [31]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Intialise models. Some are commented out as I have yet to complete hyperparamter tuning
lin = LinearRegression()
# svr = SVR()
lasso = Lasso(alpha=0.01, max_iter=100, selection='random')
# elas = ElasticNet()
dt = DecisionTreeRegressor(max_depth=5, max_features='auto', max_leaf_nodes=20,
                      min_samples_leaf=8, min_weight_fraction_leaf=0.1)
knn = KNeighborsRegressor(algorithm='kd_tree', leaf_size=10, n_neighbors=35, p=1)
gbr = GradientBoostingRegressor(learning_rate=0.01, max_depth=6, n_estimators=500,
                          subsample=0.5)
# sgd = SGDRegressor()
# ker = KernelRidge()
# bayes = BayesianRidge()

# Create dictionary of the hyper tuned models. This can be cycled through to run various models
regressor_dict = {
    'Linear' : [lin],
    'Lasso' : [lasso],
    'DecisionTree' : [dt],
    'KNeighbours' : [knn],
    'GradientBoost' : [gbr],
}

# Split data for modelling 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#### Scaling the data
Here you can scale the data to improve thee model

In [25]:
# create scaler
scaler = StandardScaler()
# fit scaler to train set - which means it identifies the values for scaling only on the training set and stores this in the scaler
scaler.fit(X_train)
# The scaler can now be applied to the train, validation and test sets which holds the info from the train set only
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#### Hyperparamater tuning
I have completed this for vairous model types and use the best model for each in the intialisation phase above.

## knn

In [26]:
from sklearn.model_selection import GridSearchCV

grid_knn = {
    'n_neighbors': [5, 35],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30],
    'p': [1, 2]
    }

gs = GridSearchCV(
    knn, 
    grid_knn, 
    cv=5,
    verbose=2
    )

gs.fit(X_train, y_train)
gs.best_estimator_


Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=uniform; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=uniform; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=uniform; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=uniform; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=uniform; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=distance; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=distance; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=distance; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=distance; total time=   0.3s
[CV] END algorithm=auto, leaf_size=10, n_neighbors=5, p=1, weights=distance; to

KNeighborsRegressor(leaf_size=10, n_neighbors=35)

## Lasso

In [ ]:
# Must scale the independent varibles for a regularisation model like this.
# Consider using r2 
# alphas = np.linspace(0, 0.2, 21)
# print(alphas)

grid_lasso = {
    'alpha': [0.01],
    'max_iter': [10, 10, 30, 40, 50, 70, 90, 100],
    'selection': ['random'],
    }

gs = GridSearchCV(
    lasso, 
    grid_lasso, 
    cv=5,
    verbose=2
    )

gs.fit(X_train, y_train)
gs.best_estimator_

Decision tree

In [ ]:
grid_dt = {
    "splitter":["best","random"],
    "max_depth" : [5, 10],
    "min_samples_leaf":[4, 5, 8, 9, 10],
    "min_weight_fraction_leaf":[0.1,0.2],
    "max_features":["auto"],
    "max_leaf_nodes":[20] 
    }

gs = GridSearchCV(
    dt, 
    grid_dt, 
    cv=5,
    verbose=2
    )

gs.fit(X_train, y_train)
gs.best_estimator_

GrdientBoost

In [58]:
grid_gbr = {
    'learning_rate': [0.01,0.02,0.03],
    'subsample'    : [0.5, 0.2, 0.1],
    'n_estimators' : [100,500],
    'max_depth'    : [4,6,8]
    }

gs = GridSearchCV(
    gbr, 
    grid_gbr, 
    cv=5,
    verbose=2
    )

gs.fit(X_train, y_train)
gs.best_estimator_

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.5; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.5; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.5; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.5; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.5; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.2; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.2; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.2; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.2; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.2; total time=   0.2

GradientBoostingRegressor(learning_rate=0.01, max_depth=6, n_estimators=500,
                          subsample=0.5)

## Run an assess models

No scaling or cross validation - The simpliest method to assess each models performance

In [ ]:
for model in regressor_dict:
    print(f'Training the {model} model')
    regressor_dict[model][0].fit(X_train, y_train)
    y_pred = regressor_dict[model][0].predict(X_test)
    y_pred_train = regressor_dict[model][0].predict(X_train)
    print('Mean Absolute Error: \n Test', metrics.mean_absolute_error(y_test, y_pred) )
    # print('Train', metrics.mean_absolute_error(y_train, y_pred_train))  
    print('Mean Squared Error: \n Test', metrics.mean_squared_error(y_test, y_pred))  
    # print('Train', metrics.mean_squared_error(y_train, y_pred_train))
    print('Root Mean Squared Error: \n Test', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    # print('Train', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
    print(df.head(5))

Scaling without cross validation - as above but using scaling

In [ ]:
# create scaler
scaler = StandardScaler()
# fit scaler to train set - which means it identifies the values for scaling only on the training set and stores this in the scaler
scaler.fit(X_train)
# The scaler can now be applied to the train, validation and test sets which holds the info from the train set only
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
# the above is peromes using scaler.transform([dataset_name])
for model in regressor_dict:
    print(f'Training the {model} model')
    regressor_dict[model][0].fit(X_train, y_train)
    y_pred = regressor_dict[model][0].predict(X_test)
    y_pred_train = regressor_dict[model][0].predict(X_train)
    print('Mean Absolute Error: \n Test', metrics.mean_absolute_error(y_test, y_pred) )
    # print('Train', metrics.mean_absolute_error(y_train, y_pred_train))  
    print('Mean Squared Error: \n Test', metrics.mean_squared_error(y_test, y_pred))  
    # print('Train', metrics.mean_squared_error(y_train, y_pred_train))
    print('Root Mean Squared Error: \n Test', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    # print('Train', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
    print(df.head(5))

Using a pipeline for each model

In [33]:
for model in regressor_dict:
    print(f'Training the {model} model')
    model = regressor_dict[model][0]
    pipe = Pipeline(
    steps= [
        ('scaler', StandardScaler()), # is this scaling the target variable too? Will this impact output and MSE
        ('model',  model)
        ]
    )
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
    # do i only put the train in or with cross validation it is OK to go with all
    #failing is it beucse im parsign a df and not an matrix
    scores = cross_val_score(pipe, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=1)
    scores = abs(scores)
    print(len(scores))
    print(scores)
    print(np.mean(scores))


Training the Linear model
30
[1.58181873e+02 6.92898664e+12 7.43541316e+12 1.43117621e+02
 1.59037975e+02 3.49508054e+13 1.64553830e+02 1.47791727e+02
 1.63154743e+02 1.01586068e+13 1.44175379e+02 8.07566220e+13
 4.19807222e+14 1.61965253e+02 3.98684822e+12 2.66481335e+13
 1.67915727e+02 1.65004726e+02 1.50511589e+02 1.54381109e+02
 1.49493889e+02 7.84062752e+14 4.50567443e+13 1.52906200e+02
 1.61792734e+02 2.49413242e+13 1.58040266e+02 2.49752131e+14
 4.61194324e+13 1.44857113e+02]
58020167371754.72
Training the Lasso model


/home/adamw/miniconda3/envs/fb_marketplace/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.449e+07, tolerance: 1.811e+04
  model = cd_fast.enet_coordinate_descent(
/home/adamw/miniconda3/envs/fb_marketplace/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.157e+07, tolerance: 1.821e+04
  model = cd_fast.enet_coordinate_descent(
/home/adamw/miniconda3/envs/fb_marketplace/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of 

30
[157.1555129  150.64871445 161.51694161 142.30720949 158.60890325
 153.32861029 163.97465664 147.21977485 159.35262698 160.79752202
 143.90023356 153.71591286 158.13594526 138.5316813  159.2058547
 165.57001529 167.05286706 162.36881704 150.42654847 154.3325656
 149.61308484 168.66622893 159.56870403 151.88891019 160.85596773
 151.62095068 156.3774876  161.06221483 144.1950317  144.31928875]
155.2106260964402
Training the DecisionTree model
30
[177.13627164 171.58475509 182.6600903  157.43197157 176.48443942
 169.63773527 185.17710103 162.54948026 185.55109258 176.82116721
 154.92791988 170.91264145 175.54700654 161.34630569 180.10021295
 189.61782942 184.20368428 184.38216498 174.34964386 169.09066165
 165.18745037 185.84015823 177.75729954 168.0897355  181.88085229
 169.27306691 184.83644224 178.03239533 170.34487887 162.15695387]
174.43038027358642
Training the KNeighbours model
30
[152.54228746 153.82380781 160.77427787 140.34366505 159.11676268
 154.20697309 164.97354103 144.96